# Group-level analyses
This week's lab is about group-level models, multiple comparison correction (MCC), and region-of-interest (ROI) analysis. In this notebook, we will focus on group-level models, which we'll demonstrate and explain using both FSL and Python. 

We'll focus on the "summary statistics" approach again, in which we'll demonstrate how we average $c\beta$-terms across runs (in run-level analyses) and subjects (in grouplevel analyses) using the GLM. Then, we're going to show you how to test more extensive hypotheses in grouplevel models. 

**What you'll learn**: after this lab, you'll ...

- understand the concept of the summary statistics approach
- be able to construct different grouplevel models (in FSL)

**Estimated time needed to complete**: 2 hours<br>

In [ ]:
# Some imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## What are group-level models?
Last week, we discussed the multilevel models and the summary statistics approach. Specifically, we focused on how data from different runs are usually (in non-fMRI contexts) are analyzed in a single multilevel GLM by "concatenating" the data. And how, in fMRI, we usually don't take this approach due to the computational burden and use the summary statistics approach, which analyzes each run separately and subsequently aggregates the data in a second, run-level GLM. 

In this notebook, we will extend this idea of analyzing data from multiple levels by looking at data from multiple subjects and how to analyze this data in group-level models. We will look at two "flavors" of group-level analyses: parametric and non-parametric.

![](https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=965&h=745)

## Parametric analyses
The most often used "flavor" of fMRI (group-level) analyses are *parametric*: it assumes that the data can be modeled using specific probability distributions. For example, we assume that the results of statistical tests of parameters (i.e., $t$-values), given that null hypothesis is true, are distributed according to the Students $t$-distribution (with a particular degrees-of-freedom):

\begin{align}
t_{c\hat{\beta}} \sim \mathcal{T}(\mathrm{df})
\end{align}

where you can read the $\sim$ symbol as "is distributed as". Importantly, the validity of the computed $p$-values depends on whether the choice of distribution is appropriate. If not, you might risk inflated type 1 or type 2 errors.

The first-level and run-level GLMs that we have discussed so far are examples of parametric analyses. There are also *non-parametric* versions of the GLM that do not assume any particular form of distribution; while somewhat more computationally expensive, this is become a more and more popular alternative to (group-level) parametric analyses. Importantly, the difference between parametric and non-parametric analyses is only important for the *inference* (not the *estimation*) aspect of the (group-level) analyses.

Now, let's focus on the parametric version of group-level analyses. Basically, this amounts to doing the same thing as we did last week with the run-level analyses, but this time, the results from our run-level analyses ($c\hat{\beta}^{*}$) across different subjects will become our target ($y^{\dagger}$). Note that we will use the "dagger" ($^{\dagger}$) superscript to denote that the mathematical terms belong to the group-level model (just like the $^{*}$ superscript in our notebooks refers to terms belonging to the run-level models).

To reiterate, the results from our run-level analyses ($c\hat{\beta}^{*}$), or first-level analyses if we only have a single run, become our dependent variable in our group-level analysis ($y^{\dagger}$):

\begin{align}
y^{\dagger} = c\hat{\beta}^{*}
\end{align}

Again, the group-level represents a GLM with a particular design matrix ($\mathbf{X}^{\dagger}$) and parameters ($\beta^{\dagger}$):

\begin{align}
y^{\dagger} = \mathbf{X}^{\dagger}\beta^{\dagger} + \epsilon^{\dagger}
\end{align}

And the group-level parameters can be estimated with OLS:

\begin{align}
\hat{\beta}^{\dagger} = (\mathbf{X}^{\dagger\ T} \mathbf{X}^{\dagger})^{-1}\mathbf{X}^{\dagger}y^{\dagger}
\end{align}

As mentioned last week, the parameter estimation procedure (i.e., estimating $\hat{\beta}^{\dagger}$) is relatively straightforward, but the inference procedure depends on the specific variance approach: fixed, random, or mixed effects. If your aim is to perform inference to the population, *you should never use a fixed-effects type of analysis across subjects*, as this will typically inflate your type 1 error greatly.

That leaves us with random effects and mixed effects GLMs. 

### Random effects
Let's first focus on a random effects type of analysis (which is commonly used in group-level analysis in the SPM software package, for example), as this is relatively easy to understand. For now, let's use simulated data. Suppose that we want to do a group-analysis of our run-level `flocBLOCKED` data. Specifically, we are interested in the difference between faces and places ($H_{0}: \beta_{face}^{*} = \beta_{house}^{*} = 0$, $H_{a}: \beta_{face}^{*} > \beta_{house}^{*}$). As such, we'll use the corresponding $c\hat{\beta}^{*}$ terms from our run-level analysis (i.e., the contrasts against baseline, COPE1 and COPE2) as our new target $y^{\dagger}$ as follows:

\begin{align}
y^{\dagger} = \begin{bmatrix}
c\hat{\beta}^{*}_{1, F>0} \\
c\hat{\beta}^{*}_{2, F>0} \\
\vdots \\
c\hat{\beta}^{*}_{N, F>0} \\
c\hat{\beta}^{*}_{1, P>0} \\
c\hat{\beta}^{*}_{2, P>0} \\
\vdots \\
c\hat{\beta}^{*}_{N, P>0}
\end{bmatrix}
\end{align}

For our simulation, we'll assume that we have 20 subjects ($N=20$).

<div class='alert alert-info'>
    <b>ToThink</b> (0 points): You might think, why compute the difference contrast at the <em>group-level</em>, while we could have computed this already in the first-level analysis (and subsequently average this in the run-level and group-level analyses)? In fact, this is mathematically (largely) equivalent. However, most people prefer to postpone this step to the group-level analysis. Can you think of a reason why?
</div>

Before we are going to simulate the data, let's construct our design-matrix, $\mathbf{X}^{*}$. Assuming we have 20 subjects and two types of inputs ($c\hat{\beta}^{*}_{F>0}$ and $c\hat{\beta}^{*}_{P>0}$, our design matrix will be:

In [ ]:
N = 20
X = np.zeros((N * 2, 2))
X[:N, 0] = 1
X[N:, 1] = 1

print(X)

<div class='alert alert-info'>
    <b>ToThink</b> (1 point): In the above design matrix, we did not specify an intercept (i.e., a vector of ones). With good reason, because our group-level GLM will crash if we did. Explain shortly why an intercept for this particular design matrix (i.e., as represented by the variable <tt>X</tt> above) is not only redundant, but will make the GLM impossible to estimate. Hint: try adding an intercept, run a GLM, and see what happens.
</div>

YOUR ANSWER HERE

We are going to simulate data for a single voxel, which shows a slight preference (in run-level $c\hat{\beta}^{*}$ estimates across subjects) for faces relative to places. Specifically, we'll assume that the voxel activates on average with $\beta^{\dagger}_{faces} = 5$ for faces, while it activates on average with $\beta^{\dagger}_{places} = 4.5$ for places:

\begin{align}
y^{*} = \mathbf{X}^{\dagger}_{faces}\cdot 5 + \mathbf{X}^{\dagger}_{places} \cdot 4.5 + \epsilon^{\dagger} 
\end{align}

where the noise, $\epsilon^{\dagger}$, is normally distributed with mean 0 and a standard deviation of 1:

\begin{align}
\epsilon^{\dagger} \sim \mathcal{N}(0, 1)
\end{align}

(The symbol $\sim$ stands for "is distributed as".) We'll implement this in code below:

In [ ]:
N = 20
beta_f = 5
beta_p = 4.5
std_noise = 1
np.random.seed(42)
noise = np.random.normal(0, std_noise, size=N * 2)

y_sim = X @ np.array([beta_f, beta_p]) + noise

And let's plot it:

In [ ]:
plt.figure(figsize=(5, 16))
plt.plot(y_sim[:N], np.arange(y_sim.size // 2), marker='o', ms=10)
plt.plot(y_sim[N:], np.arange(y_sim.size // 2, y_sim.size), marker='o', ms=10)
plt.ylim(y_sim.size, -1)
plt.legend([r"$c\hat{\beta}^{*}_{face}$", r"$c\hat{\beta}^{*}_{place}$"], fontsize=15)
plt.ylabel("Subjects", fontsize=20)
plt.xlabel(r"$c\hat{\beta}^{*}$", fontsize=20)
plt.grid()
plt.title(r"$y^{\dagger}$ (run-level estimates)", fontsize=25)
plt.show()

Note that these $c\hat{\beta}^{*}$ estimates are the only thing you need for a random effects analysis! We're simply ignoring the lower-level variance terms.

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Run a group-level random-effects GLM (using the variable <tt>X</tt> as design matrix and <tt>y_sim</tt> as dependent variable). Store the estimated parameters in a variable named <tt>gl_params</tt>. Then, compute the $t$-value for the contrast $\beta^{\dagger}_{face} > \beta^{\dagger}_{place}$ and store this in a variable named <tt>gl_tvalue</tt>.
</div>

In [ ]:
''' Implement the ToDo here. '''
from numpy.linalg import inv

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_6 import test_rfx_glm
test_rfx_glm(X, y_sim, gl_params, gl_tvalue)

### Mixed-effects (in FSL)
Proper mixed-effects models are a bit too complex to implement ourselves in Python, so we'll show you how to do it in FSL! We ran the (fixed-effects) run-level models for the `flocBLOCKED` data for you already, which we download below:

In [ ]:
import os
data_dir = os.path.join(os.path.expanduser("~"), 'NI-edu-data')

print("Downloading run-level FSL FEAT results (+- 450MB) ...")
!aws s3 sync --no-sign-request s3://openneuro.org/ds003422 {data_dir} --exclude "*" --include "derivatives/fsl/sub-*/flocBLOCKED/runlevel.gfeat/*"
print("\nDone!")

Within the `flocBLOCKED` directory, the first-level (`ses-1.feat` and `ses-2.feat`) and run-level results (`runlevel.gfeat`) are stored for each subject.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Start a remote desktop environment, open a terminal, navigate to the <tt>runlevel.gfeat</tt> directory of the task <tt>flocBLOCKED</tt> of subject <tt>sub-03</tt> (so <tt>sub-03/flocBLOCKED/runlevel.gfeat</tt>), and print the directory's contents to the terminal.

(Note that the following ToDos will be checked later using test-cells.)
</div>

You should see that the `runlevel.gfeat` directory contains multiple `cope*.feat` directories. These directories correspond to the results of the run-level (intercept-only) analyses of the seven different first-level contrasts. In contrast to what you did yourself last week (in which you concatenated the COPEs from the $c\hat{\beta}_{\mathrm{face}}$ and $c\hat{\beta}_{\mathrm{place}}$ contrasts in a single GLM), we were lazy and used the "Inputs are lower-level FEAT directories" option in FEAT. This runs the exact same GLM (using an intercept-only run-level design matrix) for all first-level contrasts separately. 

For example, the data in the `runlevel.gfeat/cope1.feat` directory corresponds to the first contrast from the first-level analysis ($\beta_{face} > 0$), the data in the `runlevel.gfeat/cope2.feat` directory corresponds to the second contrast from the first-level analysis ($\beta_{place} > 0$), etc. etc.

In our intercept-only run-level analysis, we only computed a run-level contrast reflecting the average across the two sessions, i.e., using the contrast vector $[1]$. The results of this run-level contrast, for each first-level contrast, are stored in the `cope1.nii.gz` file.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Open Feat, change "First-level analysis" to "Higher-level analysis" and change the input type to "Inputs are 3D cope images from feat directories". Then, click on the "Stats" tab.
</div>

Under the "Stats" tab, you can change the type of inference you want FEAT to do. By default, it uses a particular mixed-effects approach that FSL calls "FLAME 1". In general, we recommend using this type of inference. The other mixed-effects approach, "FLAME 1+2", is also a good choice, but may take much longer to run. The "Mixed effects: Simple OLS" is, unlike the same suggests, the random-effects option, which discards the lower-level variance estimates. Lastly, the "Randomise" option is a non-parametric (random-effects) variant, that will be discussed in more detail later.



<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Set the option to "Mixed effects: FLAME 1". Enable the "use automatic outlier de-weighting" option (should be yellow). Then, go to post-stats and set the "Thresholding" option to "Cluster" with a Z threshold of 3.7 and a "Cluster P threshold" of 0.05. This refers to a particular type of multiple comparison correction that will be discussed later.
</div>

Now, let's define the dependent variable ($y^{\dagger}$). Suppose, just like we did with the simulated data, that we have the following null and alternative hypothesis:

$H_{0}: \beta_{face}^{*} - \beta_{house}^{*} = 0$<br>
$H_{a}: \beta_{face}^{*} - \beta_{house}^{*} > 0$

Note that the contrast evaluating the difference between these two parameters was *not* computed in the first-level or run-level analyses, so we have to create this ourselves by concatenating the COPEs corresponding to the $\beta_{face}^{*} > 0$ and $\beta_{house}^{*} > 0$ run-level contrasts, just like you did last week. 

Doing this for all subjects becomes a bit cumbersome though, so you only need to do this for `sub-02`, `sub-03`, and `sub-04`. Make sure the first three inputs correspond to the face-against-baseline COPE (COPE nr. 1) and the last three inputs correspond to the place-against-baseline COPE (COPE nr. 2). 

\begin{align}
y^{\dagger} = \begin{bmatrix}
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-02}} \\
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-03}} \\
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-04}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-02}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-03}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-04}} \\
\end{bmatrix}
\end{align}

Note that you need the actual `cope*.nii.gz` files &mdash; *not* the `.feat` directory itself.

(Realize, though,  that having only 3 subjects for your group-analysis would be a hopelessly underpowered study.)

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): In the "Data" tab, use the option "Inputs are 3D cope images from FEAT directories", set the number of inputs to 6, and select the correct files. Note that within the "Select input data" window, you can copy/paste the paths using CTRL+C / CTRL+V, which might make the process a little easier/faster (not forget to actually modify the paths after copy/pasting).
</div>

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In the "Data" tab, set the output directory to your home folder + <tt>grouplevel_task-flockBLOCKED</tt>.
</div>

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): In the “Stats” tab, click on "Full model setup". Create two predictors ("EVs") that model the mean of the $c\beta^{*}_{face} > 0$ (first EV) and the $c\beta^{*}_{place} > 0$ (second EV). Then, add another predictor that models the effect of age on the $c\beta^{*}_{face}$ contrast *only*. The ages of the three participants (from sub-02 to sub-04) are: $[21, 32, 25]$. Importantly, make sure you center (de-mean) the age values before entering them in FEAT, as this improves the interpretation of the other predictors (you can compute the de-meaned age values, for example, in a new code cell).
</div>

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): In the "Contrasts & F-tests" tab, define the following group-level contrasts:<br>
    
1. $H_{0}: \beta^{\dagger}_{face} = 0$<br>$H_{a}: \beta^{\dagger}_{face} > 0$<br><br>
2. $H_{0}: \beta^{\dagger}_{place} = 0$<br>$H_{a}: \beta^{\dagger}_{place} > 0$<br><br>
3. $H_{0}: \beta^{\dagger}_{face} - \beta^{\dagger}_{place} = 0$<br>$H_{a}: \beta^{\dagger}_{face} - \beta^{\dagger}_{place} > 0$<br><br>
4. $H_{0}: \beta^{\dagger}_{age} = 0$<br>$H_{a}: \beta^{\dagger}_{age} > 0$<br><br>
5. $H_{0}: \beta^{\dagger}_{age} = 0$<br>$H_{a}: \beta^{\dagger}_{age} < 0$<br>

</div>

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Save your design (using the "Save" button in the bottom of the FEAT window) in your `week_6` directory and name it <tt>setup_feat_grouplevel.fsf</tt>.
</div>

Below, you can test your implementation.

In [ ]:
''' Tests your setup_feat.fsf file: inputs'''
import os
import os.path as op
import niedu

here = op.abspath('')
par_dir = op.basename(op.dirname(here))
home = op.expanduser("~")

fsl_deriv_dir = op.join(os.path.expanduser("~"), 'NI-edu-data', 'derivatives', 'fsl')
    
if par_dir != 'solutions':  # must be a student-account
    fsf = 'setup_feat_grouplevel.fsf'
    if not op.isfile(fsf):
        print("Couldn't find a 'setup_feat_group.fsf' file in your week_6 directory!")

    with open(fsf, 'r') as f:
        fsf = f.readlines()

    fsf = [txt.replace('\n', '') for txt in fsf if txt != '\n']
    fsf = [txt for txt in fsf if txt[0] != '#']  # remove commnts

    from niedu.tests.nii.week_6 import test_grouplevel_fsf_inputs
    test_grouplevel_fsf_inputs(fsf, fsl_deriv_dir)

In [ ]:
''' Tests your setup_feat.fsf file: Evs'''
if par_dir != 'solutions':
    from niedu.tests.nii.week_6 import test_grouplevel_fsf_evs
    test_grouplevel_fsf_evs(fsf)

In [ ]:
''' Tests your setup_feat.fsf file: contrasts'''
if par_dir != 'solutions':
    from niedu.tests.nii.week_6 import test_grouplevel_fsf_cons
    test_grouplevel_fsf_cons(fsf)

## Non-parametric analyses
*This is an optional section*

Before we go on with the topic of "multiple comparison correction" in the next notebook, let's focus on another type of inference in statistical models: permutation-based non-parametric analyses. Unlike parametric analyses (which we discussed in the previous section), non-parametric analyses do not assume anything about how our statistics-of-interest are distributed. Instead, it uses the data itself to estimate a distribution of your test-statistic under the null hypothesis. Note that this type of inference is also available in FSL, where it is called "randomise". 

Permutation tests work (roughly) as follows:
1. The statistic (e.g., $t$-value) is computed as normal;
2. The rows of design matrix ($\mathbf{X}$) are shuffled and the statistic is (re-)computed, which is iterated a number of times (e.g., 1000);
3. The ("non-parametric") $p$-value is computed

Let's do this for our previously simulated data (the variables `X` and `y_sim`).

### Computing the "observed" statistic
First, we need to compute our "observed" statistic. In the framework of the GLM, this is often a $t$-statistic (or $F$-statistic). But note that permutation-based tests work for any kind of statistics. (In fact, permutation-based tests are especially useful for statistics with unknown distributions!)

We are sure that you, by now, know how to compute $t$-values, so we are going to use a predefined function `compute_tvalue` with three arguments: `X`, `y`, and `c`. We'll compute the $t$-value for our data (`X` and `y_sim`) and store it in a variable called `t_obs` (for "observed").

In [ ]:
from niedu.utils.nii import compute_tvalue
t_obs = compute_tvalue(X, y_sim, c=np.array([1, -1]))

### Run the permutations
The second step is to, iteratively, permute the data and (re-)compute the test statistic. In most cases (assuming that the datapoints, $y$, are completely independent, i.e., there is no "autocorrelation"), permutation within the GLM framework is done by shuffling the rows of the design matrix ($\mathbf{X}$). This is usually done a larger number of times (e.g., 1000 or more).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Below, we started implementing the permutation loop. Can you finish it? You may want to use the <a href="https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.permutation.html">np.random.permutation</a> function. Make sure to save the permuted t-value each iteration (by storing it in the <tt>t_perm</tt> array).
</div>

In [ ]:
""" Implement the ToDo here. """
np.random.seed(42)
iters = 1000
t_perm = np.zeros(iters)

for i in range(iters):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
np.testing.assert_almost_equal(t_perm.mean(), 0, decimal=1)
print("Well done!")

### Calculate the p-value
The last step is to calculate the $p$-value. This is done by dividing the number of permuted statistics *larger* than the observed statistic plus one by the total number of permutations plus one. Or, mathematically, for $P$ permutations, the $p$-value for any statistic $s$ with observed value $s^{\mathrm{obs}}$ and vector of permuted values $\mathbf{s}^{\mathrm{perm}}$ is defined as:

\begin{align}
p = \frac{\sum_{i=1}^{P}\mathbf{I}(\mathbf{s}^{\mathrm{perm}}_{i} \geq s^{\mathrm{obs}}) + 1}{P + 1}
\end{align}

where $\mathbf{I}$ is an indicator function, returning $1$ when the condition is true and $0$ otherwise.

We can also visualize it:

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(t_perm)
plt.axvline(t_obs, ls='--', c='red', lw=3)
plt.legend(['observed', 'permuted'])
plt.xlabel(r'$t$-value', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.show()

In the above figure, the number of values right of the dashed red line ($t^{obs}$) plus 1 divided by the number of permutations plus 1 ($P+1$) is our $p$-value! Now, try computing it yourself!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Compute the $p$-value corresponding to our observed $t$-statistic (<tt>t_obs</tt>) and store it in a variable named <tt>p_perm</tt>.
</div>

In [ ]:
# Implement your ToDo here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. 
Note: this may differ slightly due to the exact implementation of the permutation procedure
'''
np.testing.assert_almost_equal(p_perm, 0.03, decimal=2)
print("Well done!")

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Shuffling rows works for most designs, but what if you have an intercept-only model (i.e., a one-sample $t$-test), where a design matrix consisting only of a vector of ones? Permuting the rows won't work there. An alternative for this scenario is "sign flipping": randomly changing ones to minus ones (with replacement).

Below, we simulate some new data (<tt>y_sim2</tt> and <tt>X2</tt>). Can you compute the non-parametric $p$-value for this one-sample $t$-test? Use a 1000 permutations and store the $p$-value in a new variable named <tt>p_perm2</tt>.

Hint: to generate random "signs", take a look <a href="https://stackoverflow.com/questions/46820182/randomly-generate-1-or-1-positive-or-negative-integer">here</a>.
</div>

In [ ]:
np.random.seed(17)
y_sim2 = np.random.normal(0, 1, size=100)
X2 = np.ones(100)[:, np.newaxis]

iters = 1000

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
# The p-value might slightly differ from 0.17 depending on the
# exact method of generating random signs.
np.testing.assert_almost_equal(p_perm2, 0.17, decimal=2)
print("Well done!")

### Threshold-Free Cluster Enhancement (TFCE)
Apart from (or perhaps because of) the fact that non-parametric analyses are blind to the distribution of effects, non-parametric analyses have an additional advantage: they can use a "trick" called "Threshold-Free Cluster Enhancement" (TFCE). This method allows for voxel-based inference while taking into account the spatial size of the effect. You could see TFCE as a method to "boost" high-amplitude voxels that are located within a cluster of other high-amplitude voxels. 

However, we don't know how these "boosted" values are distributed (they might or might not be $\mathcal{T}$ distributed)! Fortunately, as said before, non-parameteric analyses don't care how the computed statistic is distributed (under the null). How TFCE works mathematically is beyond the scope of the course (but you can learn more in [this video](https://www.youtube.com/watch?v=q7cWw8WC0Ws)), but in fact recommend using this technique in combination with FSL's `randomise` tool, also because it provides an elegant and effective way to control for multiple comparisons (the topic of section 3).

##  Checking out results from group-level FEAT analyses
We have run a group-level model for a couple of our lower-level contrasts. The results from the group-level analysis that we are going to take a look at will be downloaded below:

In [ ]:
print("Downloading group-level FSL FEAT results (+- 67MB) ...")
!aws s3 sync --no-sign-request s3://openneuro.org/ds003422 {data_dir} --exclude "*" --include "derivatives/fsl/grouplevel_task-flocBLOCKED/contrast-faceGTother_method-FLAME1_thresh-uncorr05.gfeat/*"
print("\nDone!")

In [ ]:
gl_dir = os.path.join(fsl_deriv_dir, 'grouplevel_task-flocBLOCKED', 'contrast-faceGTother_method-FLAME1_thresh-uncorr05.gfeat')
print(f"Data is located here: {gl_dir}")

This contains the results from the group-level analysis of the lower-level $4 \cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object} > 0$ contrast. (The "faceGTother" stands for "face greater than other conditions".)

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Open a terminal, navigate to the above directory, and print its contents to the terminal window.
</div>

You should see the usual files: HTML-reports, thresholded results (as nifti files), and files with information about the design (`design.con`, `design.fsf`, etc.). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Open the <tt>report.html</tt> file (with the command <tt>firefox report.html</tt>). Click on "Inputs", which will show you the data that we used as inputs (i.e., $y^{\dagger}$) for our group-level analysis.
</div>

As you can see, the inputs for our group-level analysis are not individual `cope*.nii.gz` files, but entire `.feat` directories. This is because we used the "Inputs are lower-level FEAT directories" option.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Click on "Results". This will show you a page with the simple "intercept-only" design (i.e., a design for a one-sample $t$-test). Now, click on "Lower-level contrast 1 (average)" to view in the group-level result from this particular run-level contrast.
</div>

On this page, you can view the results, as $z$-statistic images, from the group-level analysis plotted on a set of axial slices. Below the brain plot, you can see the "time series plot", which does not actually show a "time series", but rather the input (i.e., $y^{\dagger}$) of the voxel with the highest $z$-value. Thus, the "time series" here do not reflect datapoints over time (as in first-level analyses), but run-level $c\hat{\beta}^{*}$ estimates of different subjects!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In your terminal, open <tt>FSLeyes</tt>. Then, click on <tt>File</tt> &rarr; <tt>Add standard</tt> &rarr; select <tt>MNI152_T1_2mm_brain.nii.gz</tt> &rarr; click on <tt>Open</tt>.

Then, click on <tt>File</tt> &rarr; <tt>Add from directory</tt> &rarr; navigate to and select the <tt>contrast-faceGTothers_method-FLAME1_thresh-uncorr05.gfeat/cope1.feat</tt> directory &rarr; click <tt>Open</tt>.
</div>

This automatically opens the `filtered_func_data.nii.gz` file: a 4D nifti file with the run-level $c\hat{\beta}^{*}$ estimates of different subjects across the fourth dimension (i.e., data from different subjects are represented by different volumes, our new $y^{\dagger}$). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Set the minimum value to 100 (in the top menu) and change the colormap to "Red-Yellow". Note that the minimum value of 100 is rather arbitrary, but it removes some of the "visual clutter".

Now, go to voxel $[26, 39, 25]$, which is (roughly) located in the right fusiform gyrus (the "FFA") and, <em>for this participant</em> (<tt>sub-02</tt>), seems to activate relatively strongly in response to faces (rather than to other images). 
</div>

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Increase the volume number from 0 to 1 (in the "Location" panel), which will show you the run-level $c\hat{\beta}^{*}$ estimates from a different subject (i.e., <tt>sub-03</tt>). You'll notice that this participant's "FFA" is in a slightly different location! In fact, when you view the other volumes (i.e., the $c\hat{\beta}^{*}$ maps of other subjects), you see that the anatomical location of the effects varies substantially!
</div>

<div class='alert alert-info'>
    <b>ToThink</b> (1 point): Name two reasons why an effect (i.e., a significant "blob") may differ in anatomical location across subjects.
</div>

YOUR ANSWER HERE

Alright, now let's look at some real group-level results in FSLeyes!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In FSLeyes, delete the <tt>cope1: filtered_func_data</tt> overlay (or make it invisible by clicking on the eye icon). Then, open the <tt>thresh_zstat1.nii.gz</tt> (<tt>File</tt> &rarr; <tt>Add from file</tt>) and set the colormap to "Red-Yellow".
</div>

Here, you see the "thresholded" $z$-values plotted on the template (MNI152, 2mm) that FSL by default uses for its group-analyses. 

<div class='alert alert-warning'>
    <b>ToDo/ToThink</b> (1 point): Go to the voxel at location $[37, 60, 28]$. According to the Harvard-Oxford Subcortical structural atlas, to what anatomical brain region does this voxel most likely belong? Write this down in the text cell below. Check out the notebook from last week if you forgot how to open the Atlas panel.
</div>

YOUR ANSWER HERE

The red-yellow voxels that you see are those that have $z$-values corresponding to $p$-values smaller than 0.05 (the other voxels with $p$ > 0.05 are set to 0). This particular thresholding procedure gives what people often refer to as "uncorrected results". Although they are thresholded at a particular $p$-value, the results are referred to as "uncorrected" because they have not been corrected for multiple comparisons &mdash; the topic of the next notebook (`MCC.ipynb`).